In [1]:
# displays static in notebook
%matplotlib inline  
# displays interactive outside notebook
#%matplotlib   
# displays interactive in notebook
#%matplotlib notebook  

# to work correctly with numbers
import numpy as np

# plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# conversion between coordinate systems
import astropy.coordinates as coord
import astropy.units as u

#from astropy import units as u
from astropy.coordinates import SkyCoord

# for making data catalogues that are easily uploaded to Javascript
import json

# for loading fits files
from astropy.io import fits

# for interpolating values
import scipy as scipy
from scipy.interpolate import BSpline, make_interp_spline



folder = ''

mpl.rcParams['font.family']='serif'
mpl.rcParams['font.size']=14

In [2]:
def T_to_RGB(temperature, verbose=False):
    # assign colors using table from James Hedberg, https://sciencesims.com/sims/star-color/
    T, R, G, B = np.loadtxt('colorTable.csv', unpack = True, delimiter=',', skiprows=2)
    l, r = [(2, 0.0)], [(2, 0.0)]
    useTemp = temperature
    if (np.size(temperature) == 1):
        if (temperature < np.min(T)):
            useTemp = np.min(T)
        if (temperature > np.max(T)):
            useTemp = np.max(T)
    else:
        tooCool = useTemp < np.min(T)
        useTemp[tooCool] = np.min(T)
        tooHot = useTemp > np.max(T)
        useTemp[tooHot] = np.max(T)

    if verbose:
        print('using temperature: ', useTemp)
    T_to_R = make_interp_spline(T, R, bc_type=(l, r))
    R = T_to_R(useTemp)
    T_to_G = make_interp_spline(T, G, bc_type=(l, r))
    G = T_to_G(useTemp)
    T_to_B = make_interp_spline(T, B, bc_type=(l, r))
    B = T_to_B(useTemp)
    
    if (np.size(temperature) == 1):
        if R < 0:
            R = 0
        if R > 1: 
            R = 1
        if G < 0:
            G = 0
        if G > 1: 
            G = 1
        if B < 0:
            B = 0
        if B > 1: 
            B = 1
    else:
        outOfBounds = R < 0
        R[outOfBounds] = 0
        outOfBounds = R > 1
        R[outOfBounds] = 1
        outOfBounds = G < 0
        G[outOfBounds] = 0
        outOfBounds = G > 1
        G[outOfBounds] = 1
        outOfBounds = B < 0
        B[outOfBounds] = 0
        outOfBounds = B > 1
        B[outOfBounds] = 1

    return R, G, B

def RGB_to_Hex(R,G,B, scaledToOne=True, verbose=False):
    R0 = int(np.floor(R * 255**scaledToOne))
    G0 = int(np.floor(G * 255**scaledToOne))
    B0 = int(np.floor(B * 255**scaledToOne))
    if verbose:
        print(R0,G0,B0)
    return '%02x%02x%02x' % (R0,G0,B0)


def BV_to_T(BV, verbose=False):
    # REF?
    t = 4600 * ((1.0 / ((0.92 * BV) + 1.7)) + (1 / ((0.92 * BV) + 0.62)) )
    return t

def WriteCatalog(catalogName, params, paramNameString):
    #first, create and open the text file for writing. Choose a name: 
    catalog = open(catalogName, 'w')

    #write first row
    catalog.write('# ' + paramNameString + '\n')

    #then, loop through stars and write 1 row for each star.
    for i in range(len(params[0])): #loop over stars
        for j in range(len(params)): #loop over parameters (ra, dec, v, verr,...)
            catalog.write(str(params[j][i]))
            if j < len(params)-1:
                catalog.write(', ')
        if i < len(params[0])-1:
            catalog.write('\n')
    catalog.close()

    return

    

In [3]:
dataFile = 'HIP_Vmag_less_than_6.tsv'

# load columns from data table
HIP, Vmag, Plx, pmRA, pmDE, e_Plx, BV, RA, DEC = np.loadtxt(dataFile, unpack = True, delimiter='|', skiprows=53)

# calculate absolute Vmagnitude
Vabs = Vmag + 5*(np.log10(Plx / 1000)+1)
# approximate relative luminosities using absolute V magnitude.  Could be improved by using a color term
relLum = 10**((Vabs - np.nanmax(Vabs)) / (-2.5))
# approximate temperatures based on BV color
temperatures = BV_to_T(BV)
# approximate relative sizes using Stefan-Boltzman
relSize = (relLum / temperatures**4)**0.5
relSize = relSize / np.nanmin(relSize)
# relative brightnesses as seen from Earth, i.e., ignoring distance
relBrightness = 10**((Vmag - np.nanmax(Vmag)) / (-2.5))
# approximate relative surface brightness
relSurfBright = relLum / relSize**2

print('# entries: ', len(RA))
print('Vmag limits: ', np.min(Vmag), np.max(Vmag))
print('parallax: ', np.min(Plx), np.max(Plx))
print('BV: ', np.min(BV), np.max(BV))
print('Vabs: ', np.nanmax(Vabs), np.nanmin(Vabs))
print('relative luminosity: ', np.nanmin(relLum), np.nanmax(relLum))
print('temperatures: ', np.min(temperatures), np.max(temperatures))
print('relative sizes: ',np.nanmin(relSize), np.nanmax(relSize))
print('relative brightness from Earth: ',np.nanmin(relBrightness), np.nanmax(relBrightness))
print('relative surface brightness: ',np.nanmin(relSurfBright), np.nanmax(relSurfBright))
print('')

# select points with Vmag brighter than 4
usePoints = (Plx > 0) & (Vmag <= 4)
print('Restricting to Vmag <= 4')
print('# entries: ', len(RA[usePoints]))
print('Vmag limits: ', np.min(Vmag[usePoints]), np.max(Vmag[usePoints]))
print('parallax: ', np.min(Plx[usePoints]), np.max(Plx[usePoints]))
print('BV: ', np.min(BV[usePoints]), np.max(BV[usePoints]))
print('Vabs: ', np.max(Vabs[usePoints]), np.min(Vabs[usePoints]))
print('relative luminosity: ', np.min(relLum[usePoints]), np.max(relLum[usePoints]))
print('temperatures: ', np.min(temperatures[usePoints]), np.max(temperatures[usePoints]))
print('relative sizes: ',np.min(relSize[usePoints]), np.max(relSize[usePoints]))
print('relative brightness from Earth: ',np.nanmin(relBrightness[usePoints]), np.nanmax(relBrightness[usePoints]))
print('relative surface brightness: ',np.nanmin(relSurfBright[usePoints]), np.nanmax(relSurfBright[usePoints]))
print('')

#check outputs
#for i in np.arange(np.sum(usePoints)):
#    print(i, RA[usePoints][i], DEC[usePoints][i], Plx[usePoints][i], Vmag[usePoints][i], \
#          BV[usePoints][i], temperatures[usePoints][i], relLum[usePoints][i], relSize[usePoints][i], \
#          relSurfBright[usePoints][i])

WriteCatalog('starProperties.csv', \
             [RA[usePoints], DEC[usePoints], Plx[usePoints], Vmag[usePoints], BV[usePoints], \
              temperatures[usePoints], relLum[usePoints], relSize[usePoints], relSurfBright[usePoints]], \
            'RA, Dec, Plx, Vmag, BV, temperature, relative Luminosity, relative Size, relative SurfaceBrightnes')

# entries:  5039
Vmag limits:  -1.44 6.0
parallax:  -1.96 742.12
BV:  -0.274 3.271
Vabs:  7.490392854746413 -inf
relative luminosity:  1.0 inf
temperatures:  2244.241517944897 15679.689085545915
relative sizes:  1.0 inf
relative brightness from Earth:  1.0 946.237161365793
relative surface brightness:  0.05750385480483357 137.01508234834463

Restricting to Vmag <= 4
# entries:  517
Vmag limits:  -1.44 4.0
parallax:  0.11 742.12
BV:  -0.269 1.865
Vabs:  6.182055686568412 -10.953036574208875
relative luminosity:  3.3368360380419815 23843596.930079106
temperatures:  3316.0297517104714 15515.240213057567
relative sizes:  1.5056916058269745 1524.5689842742588
relative brightness from Earth:  6.309573444801933 946.237161365793
relative surface brightness:  0.27408941424036576 131.3568134897944



<ipython-input-3-3c603c66ddcc>:7: RuntimeWarning: divide by zero encountered in log10
  Vabs = Vmag + 5*(np.log10(Plx / 1000)+1)
<ipython-input-3-3c603c66ddcc>:7: RuntimeWarning: invalid value encountered in log10
  Vabs = Vmag + 5*(np.log10(Plx / 1000)+1)
<ipython-input-3-3c603c66ddcc>:18: RuntimeWarning: invalid value encountered in true_divide
  relSurfBright = relLum / relSize**2


## generate parameters for aframe

In [4]:
# making a to-scale representation of each star will be difficult, since the size, 
#   surface brightness, and overall luminosity all have ~3 to 7 orders of magnitude variation
#   across the V <=4 sample
#   compared to the entire HIPARCHOS V <= 6 sample, these stars have
#   relative sizes:  1.5056916058269745 1524.5689842742588
#   relative surface brightness:  0.27408941424036576 131.3568134897944
#   relative luminosity:  3.3368360380419815 23843596.930079106
    
    
###################################################################    
# this is an arbitrary attempt to scale displayed size and brightness for display
# need to research how this is done in general
# or at the very least tweak to get a better in-app appearance

#scaleSize = 20 * (4*logLum + 1)
#scaleSize = (-1 * (Vabs[res] - np.max(Vabs[res]))+1)/4
#scaleBright = 1 - 0.5 * logLum / np.max(logLum)

# scale radius and surface brightness logarithmically
# this will mess up relative luminosities
# there's probably a better way to do this
minSize, maxSize = np.min(relSize[usePoints]), np.max(relSize[usePoints])
minSB, maxSB = np.nanmin(relSurfBright[usePoints]), np.nanmax(relSurfBright[usePoints])
print(minSize, maxSize , np.log10(maxSize/minSize), minSB, maxSB, np.log10(maxSB/minSB))

#displaySize = (np.log10(relSize / minSize) + 1) / 10
displaySize = np.log10(relSize / minSize) * (0.1 / np.log10(maxSize / minSize)) + 0.1
displayBright = np.log10(relSurfBright / minSB) * (0.5 / np.log10(maxSB / minSB)) + 0.5

#check outputs
#print('i,    plx,     Vmag,      BV,    Vabs,   size,   bright ')
#for i in np.arange(np.sum(usePoints)):
#    print(i, Plx[usePoints][i], Vmag[usePoints][i], BV[usePoints][i], Vabs[usePoints][i], displaySize[usePoints][i], displayBright[usePoints][i])
 

    
####################################################################    
#calculate colors to for display in aframe
#  T_to_RGB calculates colors with one term always at 1
#  displayBright scales according to surface brightness
HIP_R, HIP_G, HIP_B = T_to_RGB(temperatures) * displayBright
#check outputs
#print('i,    BV,    Vabs,   size,    bright   ,    R,    G,    B')
#for i in np.arange(np.sum(usePoints)):
#    print(i, BV[usePoints][i], Vabs[usePoints][i], displaySize[usePoints][i], displayBright[usePoints][i], HIP_R[usePoints][i], HIP_G[usePoints][i], HIP_B[usePoints][i])


# convert RGB to Hex
#  hrm, some items in the full dataset are crashing here.  Go ahead and restrict to usePoints
#  use a numpy array to store the color strings, to simplify later code
hexColors = np.array([])
for i in np.arange(np.sum(usePoints)):
    hexColors = np.append(hexColors, RGB_to_Hex(HIP_R[usePoints][i],HIP_G[usePoints][i], HIP_B[usePoints][i]))
#    hexColors.append(RGB_to_Hex(HIP_R[usePoints][i],HIP_G[usePoints][i], HIP_B[usePoints][i]))
#check outputs
#print('i,    BV,    bright   ,    R,    G,    B, hexColor')
#for i in np.arange(np.sum(usePoints)):
#    print(i, BV[usePoints][i], displayBright[usePoints][i], HIP_R[usePoints][i], HIP_G[usePoints][i], HIP_B[usePoints][i], hexColors[i])


# convert RA, Dec, and parallax to X, Y, Z
# calculate earth-centric XYZ
# X-axis is along RA = 0, Dec = 0
# Y-axis is along RA = 90, Dec = 0
# Z-axis is along Dec = 90

X = 1000. / Plx * np.cos(RA * np.pi/180) * np.cos(DEC * np.pi/180)
Y = 1000. / Plx * np.sin(RA * np.pi/180) * np.cos(DEC * np.pi/180)
Z = 1000. / Plx * np.sin(DEC * np.pi/180)

X = X.flatten()
Y = Y.flatten()
Z = Z.flatten()


1.5056916058269745 1524.5689842742588 3.005411051033679 0.27408941424036576 131.3568134897944 2.680560341865614


<ipython-input-4-5409e8567320>:67: RuntimeWarning: divide by zero encountered in true_divide
  X = 1000. / Plx * np.cos(RA * np.pi/180) * np.cos(DEC * np.pi/180)
<ipython-input-4-5409e8567320>:68: RuntimeWarning: divide by zero encountered in true_divide
  Y = 1000. / Plx * np.sin(RA * np.pi/180) * np.cos(DEC * np.pi/180)
<ipython-input-4-5409e8567320>:69: RuntimeWarning: divide by zero encountered in true_divide
  Z = 1000. / Plx * np.sin(DEC * np.pi/180)


In [5]:
# for output, sort by distance from close to far - this will make manual experimentation easier
sortem = np.argsort(1000.0/Plx[usePoints])
#print(sortem)

#check outputs
#print('i,    RA,   Dec,   Plx,    X,   Y,   Z')
#for i in np.arange(len(sortem)):
#    print(i, RA[usePoints][sortem][i], DEC[usePoints][sortem][i], Plx[usePoints][sortem][i], X[usePoints][sortem][i], Y[usePoints][sortem][i], Z[usePoints][sortem][i])
 

#check outputs
#print('i,      RA,          Dec,      Plx,             size,     color')
#for i in np.arange(len(sortem)):
#    print(i, RA[usePoints][sortem][i], DEC[usePoints][sortem][i], Plx[usePoints][sortem][i], displaySize[usePoints][sortem][i], hexColors[sortem][i])    


In [6]:
# aframe uses axes arranged like a computer screen, X is horizontal, Y is vertical, Z goes into screen
# so need to swap the Y and Z from my original calculation 

for i in np.arange(len(X[usePoints][sortem])):
    print('     <a-sphere') 
    print('       static-body')
    print('       position="',\
          Y[usePoints][sortem][i],\
          ' ',\
          Z[usePoints][sortem][i],\
          ' ',\
          X[usePoints][sortem][i],\
          '"') 
    print('       material="color: #',hexColors[sortem][i],'"')
    print('       radius="',displaySize[usePoints][sortem][i],'"')
    print('     ></a-sphere>') 


     <a-sphere
       static-body
       position=" -0.42125100876918153   -1.1766442218821473   -0.5037736414631858 "
       material="color: # aa9789 "
       radius=" 0.1094097935475106 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -0.42115726420752364   -1.1766806828544818   -0.5037668592842526 "
       material="color: # a1876c "
       radius=" 0.10348843993852994 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 2.476774158942196   -0.7584977167238782   -0.494330924266708 "
       material="color: # 899ada "
       radius=" 0.11166258138886648 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 2.5428248470449697   -0.5288144478121776   1.900014657880408 "
       material="color: # a2886e "
       radius=" 0.1 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 3.1609817424612596   0.3184939932046084   -1.4622856859963167 "
       material="color: # b0acb9 "
       radius=" 0.1150519153270262

       position=" 13.956737662486434   -26.773874519012903   -2.9787471484025487 "
       material="color: # 9ba2c8 "
       radius=" 0.12240112098177784 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 5.534121647805157   26.457522983820255   14.082354505470162 "
       material="color: # 959fcd "
       radius=" 0.12452950027027296 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 10.794981804993379   -27.663660689291227   7.267634561931478 "
       material="color: # 977551 "
       radius=" 0.1354127771840301 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -11.099856283248869   28.42516309761728   3.6363030954104603 "
       material="color: # 9d8060 "
       radius=" 0.13858865981206916 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -9.101633436521546   23.411092591241687   -17.951824361977664 "
       material="color: # 8297e6 "
       radius=" 0.12118196502272285 "
     ></a-sphere>
   

       material="color: # 8799dd "
       radius=" 0.1175149614723926 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 20.719634308229775   -44.38438747963683   10.282529465224012 "
       material="color: # a0856a "
       radius=" 0.14277882069300502 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -3.321946018935807   -15.237798582066189   -48.090489334995546 "
       material="color: # 8197e7 "
       radius=" 0.12311288479923821 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -47.11376361935512   1.7635798241399334   -19.295459374769795 "
       material="color: # 8a9ad9 "
       radius=" 0.11989945187299975 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -36.917014666304176   -33.905490864162786   14.150099946614073 "
       material="color: # 8197e7 "
       radius=" 0.1144444217314859 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -11.803319553261947   -38.0792

       radius=" 0.12917825995226673 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -88.3333818637645   67.39613398033137   -17.55167523851898 "
       material="color: # 8c6238 "
       radius=" 0.1626089079175087 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 86.17999165420261   46.04422814184508   56.24188026298678 "
       material="color: # 8397e5 "
       radius=" 0.13378182106962677 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 17.330509561862808   -106.68770425635009   -34.659948855017525 "
       material="color: # 8398e3 "
       radius=" 0.1314382964696898 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 96.13779380493537   24.548821886134803   55.1253553468749 "
       material="color: # 8297e6 "
       radius=" 0.12961458988816224 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 86.209034641412   46.42533741572071   57.67073376096555 "
       material="c

       material="color: # ae9f97 "
       radius=" 0.16288338524373475 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -179.8533791032057   250.45199049478074   187.76416616128233 "
       material="color: # 87592e "
       radius=" 0.17768531231208243 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 317.7753413199422   -174.80110932464416   -87.70764862427748 "
       material="color: # 835428 "
       radius=" 0.18097613555749587 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" -114.97341999019085   -244.16502229970953   -274.0320212383614 "
       material="color: # 95724c "
       radius=" 0.1723163942330706 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 367.0728495900381   37.71102514718317   113.61738345457975 "
       material="color: # 8299ed "
       radius=" 0.14294024356687032 "
     ></a-sphere>
     <a-sphere
       static-body
       position=" 372.1571460908557   -120.498294487

# to use the above block of code:

1. copy-paste into a text editor and find-replace '# ' to '#'
2. copy-paste the corrected markdown into the solar neighborhood template

In [7]:
# sample code in case you want to display stars in galactic coordinates
# have to remember to offset X coords by galcen_distance

coords = coord.ICRS(ra=RA[usePoints]*u.degree, 
                    dec=DEC[usePoints]*u.degree, 
                    distance=1000./Plx[usePoints]*u.pc, 
                    pm_ra_cosdec=np.zeros(np.sum(usePoints))*u.mas/u.yr, 
                    pm_dec=np.zeros(np.sum(usePoints))*u.mas/u.yr, 
                    radial_velocity=np.zeros(np.sum(usePoints))*u.km/u.s)

gal_coords = coords.transform_to(coord.Galactocentric)
print(gal_coords.galcen_distance * 1000 * u.pc / u.kpc)

print(gal_coords)

8122.0 pc
<Galactocentric Coordinate (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in pc
    [(-8131.27308268,  2.32274700e+01,  4.68259061e+00),
     (-8129.68034685,  1.47826714e+01,  1.98650247e+01),
     (-8109.9082785 , -8.73164494e+00, -1.94908440e+01),
     (-8145.47800839,  6.60808908e+01, -5.34593972e+01),
     (-8126.8842965 ,  2.97252423e+01, -6.27433981e+01),
     (-8118.70953218, -4.71859347e+00,  1.60086250e+01),
     (-8116.79309997, -4.64773623e+00, -1.67078475e+00),
     (-8117.0145223 , -4.20996961e+00, -2.02145857e+00),
     (-8214.6277214 ,  1.55457815e+02, -7.34298984e+00),
     (-8135.14464258,  2.28607612e+01,  4.39965350e+00),
     (-8158.30008579,  5.94425272e+01,  1.32004470e+01),
     (-8123.77789845,  4.43122789e+00, -8.18496085e+00),
     (-8125.17112555,  4.99427603e+00,  2.02836678e+01),
     (-8225.8756167 ,  1.56496